<a href="https://colab.research.google.com/github/arunsechergy/TSAI-DeepNLP-END/blob/main/assignments/assignment14/Tokenize_Python_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
file = 'drive/MyDrive/datasets/english_python_data_tabs.txt'

In [5]:
mkdir data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!cp -r  'drive/MyDrive/datasets/english_python_data_tabs.txt' 'data/'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
#from torchtext.data import Field, BucketIterator
from torchtext.legacy.data import Example, Field, BucketIterator, Dataset

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [13]:
import tokenize
import os
import re
import pandas as pd
from io import BytesIO

In [11]:
DATA_DIR = 'data'
corpus_name = "english_python_data_tabs.txt"
# corpus_name = "english_python_data.txt"
corpus = os.path.join(DATA_DIR, corpus_name)

def readLines(file, n=None):
    with open(file, 'r') as datafile:
        lines = datafile.readlines()
    return lines
    # n = len(lines) if n is None else n
    # for line in lines[:n]:
    #     print(line)

input_file= readLines(corpus)

In [14]:
q_c = {}
prev_value = []
for id, line in enumerate(input_file):
  #if line.startswith('#'):
  #if (line.lower().startswith('# write'))  or (line.lower().startswith('#write')) or (line.lower().startswith('#python')) or (line.lower().startswith('# python')):
  if re.match(r'[#]\d*\s*(Write|write|python|Python)', line):  
    q_c[line] = id
    prev_line = line
    prev_value = []
  else:
    prev_value.append(line)
    q_c[prev_line] = prev_value

In [15]:
question_code_pairs = pd.DataFrame.from_dict(q_c.items())
question_code_pairs.columns = ['question', 'code']

In [16]:
question_code_pairs['code'] = question_code_pairs['code'].apply(lambda x: "".join(x))

In [17]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [18]:
spacy_en = spacy.load('en')

In [21]:
question_code_pairs['code']

0       num1 = 5465461\nnum2 = 8765468\nsum = num1 + n...
1       def add_two_numbers(num1, num2):\n\tsum = num1...
2       num1 = 123\nnum2 = 125\nnum3 = 148\nif (num1 >...
3       num1 = 100\nnum2 = 200\nnum3 = 300\nif (num1 <...
4        def merge_lists(l1, l2):\n\treturn l1 + l2\n\n\n
                              ...                        
2130      a = 60\nb = 13\n\nc = a | b\nprint("OR", c)\n\n
2131     a = 60\nb = 13\n\nc = a ^ b\nprint("XOR", c)\n\n
2132    a = 60\n\nc = ~a\nprint("Binary Ones Complemen...
2133        c = a << 2\nprint("Binary Left Shift", c)\n\n
2134         c = a >> 2\nprint("Binary Right Shift", c)\n
Name: code, Length: 2135, dtype: object

In [37]:
question_code_pairs['code'][1]

'def add_two_numbers(num1, num2):\n\tsum = num1 + num2\n\treturn sum\n\n\n'

In [39]:
# Test Tokenization
s=question_code_pairs['code'][1]

tokens = tokenize.tokenize(BytesIO(s.encode('utf-8')).readline)
print([token.string for token in tokens])

['utf-8', 'def', 'add_two_numbers', '(', 'num1', ',', 'num2', ')', ':', '\n', '\t', 'sum', '=', 'num1', '+', 'num2', '\n', 'return', 'sum', '\n', '\n', '\n', '', '']


In [44]:
def tokenize_en(text, max_length=200):
    """
    Tokenizes English text from a string into a list of strings
    """
    tokens = [tok.text for tok in spacy_en.tokenizer(text)]
    tokens = tokens[:max_input_length]
    return 

In [45]:
def tokenize_py(text, max_length=200):
  """
  Tokenizes Python code from a string into a list of strings
  """
  tokens = tokenize.tokenize(BytesIO(text.encode('utf-8')).readline)
  tokens = [token.string for token in tokens]
  tokens = tokens[:max_input_length]
  return tokens

In [46]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize_py, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True
            )